In [ ]:
!pip install -q -U \
  "bitsandbytes==0.46.0" \
  "transformers==4.41.2" \
  "peft==0.11.1" \
  "accelerate==0.31.0" \
  "datasets==2.19.2" \
  "trl==0.8.6" \
  "huggingface_hub" \
  "minijinja" \
  "triton==3.2.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.

In [ ]:
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, get_peft_model, PeftModel
from trl import SFTTrainer
from huggingface_hub import login
import os

In [ ]:
import random
import numpy as np
import torch

def fix_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
from huggingface_hub import login
import os

login()

In [ ]:
MODEL_ID_FINETUNE = "meta-llama/Meta-Llama-3-8B-Instruct"

NEW_MODEL_NAME = f"{MODEL_ID_FINETUNE.split('/')[-1]}-synthetic-qlora"
OUTPUT_DIR = f"./{NEW_MODEL_NAME}-results"

print(f"Modelo base para fine-tuning: {MODEL_ID_FINETUNE}")
print(f"Nome do novo adaptador: {NEW_MODEL_NAME}")

Modelo base para fine-tuning: meta-llama/Meta-Llama-3-8B-Instruct
Nome do novo adaptador: Meta-Llama-3-8B-Instruct-synthetic-qlora


In [ ]:
import torch

# Configuração de Quantização
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",       # Tipo de quantização em NormalFloat 4
    bnb_4bit_compute_dtype=torch.bfloat16, # Dtype para cômputo (float16, bfloat16)
    bnb_4bit_use_double_quant=False, # Usar quantização dupla (economiza um pouco mais de memória)
)

In [ ]:
print(f"Carregando modelo base: {MODEL_ID_FINETUNE}")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID_FINETUNE,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
print("Modelo carregado.")


tokenizer = AutoTokenizer.from_pretrained(MODEL_ID_FINETUNE, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("Tokenizer carregado.")
print(f"PAD token ID: {tokenizer.pad_token_id}, EOS token ID: {tokenizer.eos_token_id}")

Carregando modelo base: meta-llama/Meta-Llama-3-8B-Instruct


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Tokenizer carregado.
PAD token ID: 128009, EOS token ID: 128009


In [ ]:
print("\nRealizando teste de inferência...")


messages = [
    {"role": "system", "content": "Você é um assistente prestativo e educado."},
    {"role": "user", "content": "Qual a capital da França?"},
]

# Aplica o template de chat do tokenizer do Llama 3
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True, # Adiciona o token de início de geração para o assistente
    return_tensors="pt"
).to(model.device) # Envia para a GPU

# Geração da resposta
outputs = model.generate(
    input_ids,
    max_new_tokens=50, # Número máximo de novos tokens a serem gerados
    do_sample=True, # Amostragem para respostas mais criativas
    temperature=0.7, # Controla a aleatoriedade
    top_k=50, # Filtra os 50 tokens mais prováveis
    top_p=0.95, # Amostragem cumulativa
    eos_token_id=tokenizer.eos_token_id # Define o token de parada
)

# Decodifica a resposta, pulando o input original
response = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)

print("\n--- Resposta do Modelo ---")
print(response)
print("\n--- Fim do Teste ---")


Realizando teste de inferência...


NameError: name 'tokenizer' is not defined

In [ ]:
import json
import os
from datasets import load_dataset
from tqdm import tqdm  # Para uma barra de progresso visual

def evaluate_model(model,
                   tokenizer,
                   dev_dataset_path,
                   output_log_file,
                   num_examples=None,
                   max_new_tokens=150):
    """
    Avalia um modelo em um número configurável de exemplos do dataset de desenvolvimento.

    Args:
        model: O modelo a ser testado (neste caso, o modelo base).
        tokenizer: O tokenizer correspondente.
        dev_dataset_path (str): Caminho para o arquivo .jsonl formatado do dev.
        output_log_file (str): Arquivo onde os resultados serão salvos.
        num_examples (int, optional): Número de exemplos a serem testados. Se None, testa todos.
        max_new_tokens (int, optional): Número máximo de tokens a serem gerados.
    """
    print(f"Iniciando avaliação do modelo base...")
    print(f" - Dataset: {dev_dataset_path}")
    print(f" - Log de saída: {output_log_file}")

    # Carrega o dataset formatado
    dev_dataset = load_dataset('json', data_files=dev_dataset_path, split='train')

    # Seleciona um subconjunto de exemplos, se especificado
    if num_examples:
        print(f" - Testando em {num_examples} exemplos (de {len(dev_dataset)}).")
        dev_dataset = dev_dataset.select(range(num_examples))
    else:
        print(f" - Testando em todos os {len(dev_dataset)} exemplos.")

    # Limpa o arquivo de log antigo, se existir
    if os.path.exists(output_log_file):
        os.remove(output_log_file)
        print(f"Arquivo de log antigo '{output_log_file}' removido.")

    # Loop principal de avaliação
    for i, example in enumerate(tqdm(dev_dataset, desc="Gerando Respostas")):
        try:
            # Ponto crucial: Extrai apenas a parte do prompt que o modelo deve ver.
            # O prompt completo contém a resposta SQL, então removemos a parte do 'assistant'.
            full_prompt = example['text']
            stop_sequence = "<|start_header_id|>assistant<|end_header_id|>"
            prompt_for_inference = full_prompt.split(stop_sequence)[0] + stop_sequence

            # Tokeniza o input
            inputs = tokenizer(prompt_for_inference, return_tensors="pt").to(model.device)

            # Gera a resposta
            with torch.no_grad(): # Desativa o cálculo de gradientes para economizar memória
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=max_new_tokens,
                    pad_token_id=tokenizer.eos_token_id,
                    eos_token_id=tokenizer.eos_token_id # Adicionado para consistência
                )

            # Decodifica apenas a parte gerada
            generated_sql = tokenizer.decode(outputs[0][len(inputs['input_ids'][0]):], skip_special_tokens=True)

            # Prepara o registro do resultado
            result_log = {
                "index": i,
                "full_original_prompt": full_prompt,
                "prompt_sent_to_model": prompt_for_inference,
                "generated_sql": generated_sql.strip()
            }

            # Mecanismo de Robustez: Salva o progresso a cada iteração.
            with open(output_log_file, 'a', encoding='utf-8') as f:
                f.write(json.dumps(result_log, ensure_ascii=False) + '\n')

        except Exception as e:
            print(f"\n❌ Erro ao processar o exemplo {i}. Pulando para o próximo.")
            print(f"   Erro: {e}")
            error_log = {"index": i, "error": str(e)}
            with open(output_log_file, 'a', encoding='utf-8') as f:
                f.write(json.dumps(error_log, ensure_ascii=False) + '\n')

    print(f"\n🎉 Avaliação concluída! Resultados salvos em '{output_log_file}'.")

In [ ]:
# Parâmetros de Configuração da Avaliação

# Caminho para o seu arquivo de desenvolvimento formatado
DEV_FILE = 'dev_formatted.jsonl'

# Nome do arquivo para salvar os logs de saída da avaliação do modelo base
LOG_FILE_BASE_MODEL = 'evaluation_results_base_model.jsonl'


NUM_EXAMPLES_TO_TEST = 200


# Chama a função de avaliação passando o modelo base

evaluate_model(
    model=model,
    tokenizer=tokenizer,
    dev_dataset_path=DEV_FILE,
    output_log_file=LOG_FILE_BASE_MODEL,
    num_examples=NUM_EXAMPLES_TO_TEST
)